In [1]:
# working on CXR8 dataset
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score,recall_score,precision_score
from sklearn.model_selection import train_test_split


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import cv2



In [2]:
# load data from images
data_dir = 'images'
images = os.listdir(data_dir)


In [3]:
images

['00000001_000.png',
 '00000001_001.png',
 '00000001_002.png',
 '00000002_000.png',
 '00000003_000.png',
 '00000003_001.png',
 '00000003_002.png',
 '00000003_003.png',
 '00000003_004.png',
 '00000003_005.png',
 '00000003_006.png',
 '00000003_007.png',
 '00000004_000.png',
 '00000005_000.png',
 '00000005_001.png',
 '00000005_002.png',
 '00000005_003.png',
 '00000005_004.png',
 '00000005_005.png',
 '00000005_006.png',
 '00000005_007.png',
 '00000006_000.png',
 '00000007_000.png',
 '00000008_000.png',
 '00000008_001.png',
 '00000008_002.png',
 '00000009_000.png',
 '00000010_000.png',
 '00000011_000.png',
 '00000011_001.png',
 '00000011_002.png',
 '00000011_003.png',
 '00000011_004.png',
 '00000011_005.png',
 '00000011_006.png',
 '00000011_007.png',
 '00000011_008.png',
 '00000012_000.png',
 '00000013_000.png',
 '00000013_001.png',
 '00000013_002.png',
 '00000013_003.png',
 '00000013_004.png',
 '00000013_005.png',
 '00000013_006.png',
 '00000013_007.png',
 '00000013_008.png',
 '00000013_00

In [4]:
# list of test images in CXR8 test_list.txt file
test_images = pd.read_csv('dataset/test_list.txt', header=None)

In [5]:
test_images

,0
0,00000003_000.png
1,00000003_001.png
2,00000003_002.png
3,00000003_003.png
4,00000003_004.png
...,...
25591,00030800_000.png
25592,00030802_000.png
25593,00030803_000.png
25594,00030804_000.png


In [6]:
train_images = pd.read_csv('dataset/train_val_list.txt', header=None)

In [7]:
train_images

,0
0,00000001_000.png
1,00000001_001.png
2,00000001_002.png
3,00000002_000.png
4,00000004_000.png
...,...
86519,00030789_000.png
86520,00030793_000.png
86521,00030795_000.png
86522,00030801_000.png


In [8]:
# read the labels from the data
labels = pd.read_csv('dataset/Data_Entry_2017_v2020.csv')

In [9]:
labels.describe()

,Follow-up #,Patient ID,Patient Age,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
count,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000
mean,8.573751,14346.381743,46.626365,2646.078844,2486.438842,0.155649,0.155649
std,15.406320,8403.876972,16.602680,341.246429,401.268227,0.016174,0.016174
min,0.000000,1.000000,0.000000,1143.000000,966.000000,0.115000,0.115000
25%,0.000000,7310.750000,34.000000,2500.000000,2048.000000,0.143000,0.143000
50%,3.000000,13993.000000,49.000000,2518.000000,2544.000000,0.143000,0.143000
75%,10.000000,20673.000000,59.000000,2992.000000,2991.000000,0.168000,0.168000
max,183.000000,30805.000000,95.000000,3827.000000,4715.000000,0.198800,0.198800


In [10]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112120 entries, 0 to 112119
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Image Index                  112120 non-null  object 
 1   Finding Labels               112120 non-null  object 
 2   Follow-up #                  112120 non-null  int64  
 3   Patient ID                   112120 non-null  int64  
 4   Patient Age                  112120 non-null  int64  
 5   Patient Gender               112120 non-null  object 
 6   View Position                112120 non-null  object 
 7   OriginalImage[Width          112120 non-null  int64  
 8   Height]                      112120 non-null  int64  
 9   OriginalImagePixelSpacing[x  112120 non-null  float64
 10  y]                           112120 non-null  float64
dtypes: float64(2), int64(5), object(4)
memory usage: 9.4+ MB


In [11]:
# check for missing values
labels.isnull().sum()


Image Index                    0
Finding Labels                 0
Follow-up #                    0
Patient ID                     0
Patient Age                    0
Patient Gender                 0
View Position                  0
OriginalImage[Width            0
Height]                        0
OriginalImagePixelSpacing[x    0
y]                             0
dtype: int64

In [12]:
# list all finding labels  in the dataset seperated by '|'
all_labels = '|'.join(labels['Finding Labels'].unique())
all_labels = all_labels.split('|')
all_labels = list(set(all_labels))




In [13]:
# one hot encode the labels
for label in all_labels:
    labels[label] = labels['Finding Labels'].apply(lambda x: 1 if label in x else 0)

In [14]:
labels

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Emphysema,Pneumothorax,Infiltration,Effusion,Hernia,Atelectasis,Pneumonia,Mass,Edema,Fibrosis
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,...,0,0,0,0,0,0,0,0,0,0
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,...,1,0,0,0,0,0,0,0,0,0
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,...,0,0,0,1,0,0,0,0,0,0
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,...,0,0,0,0,0,0,0,0,0,0
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,38,M,PA,2048,2500,0.168,...,0,0,0,0,0,0,1,1,0,0
112116,00030802_000.png,No Finding,0,30802,28,M,PA,2048,2500,0.168,...,0,0,0,0,0,0,0,0,0,0
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,...,0,0,0,0,0,0,0,0,0,0
112118,00030804_000.png,No Finding,0,30804,29,F,PA,2048,2500,0.168,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# create a dataframe with image names and labels
# make tensor of the labels
tenso = torch.tensor(labels[all_labels].values).float()
data = pd.DataFrame()
data['Image Index'] = labels['Image Index']
data[all_labels] = tenso

In [16]:
data = data.drop(columns=['No Finding'])

In [17]:
all_labels

all_labels_without_no_finding = all_labels.copy()
all_labels_without_no_finding.remove('No Finding')
all_labels_without_no_finding

['Pleural_Thickening',
 'Consolidation',
 'Cardiomegaly',
 'Nodule',
 'Emphysema',
 'Pneumothorax',
 'Infiltration',
 'Effusion',
 'Hernia',
 'Atelectasis',
 'Pneumonia',
 'Mass',
 'Edema',
 'Fibrosis']

In [18]:
# create a pytorch dataset
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class CXR8Dataset(Dataset):
    def __init__(self, data, data_dir, transform=None):
        super().__init__()
        self.data = data
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_name = self.data.iloc[index, 0]
        img_path = os.path.join(self.data_dir, img_name)
        img = Image.open(img_path)
        img = np.array(img)
        
        if len(img.shape) == 2:
            img = img[:, :, np.newaxis]
            img = np.concatenate([img, img, img], axis=2)
        if len(img.shape)>2:
            img = img[:,:,0]
            img = img[:, :, np.newaxis]
            img = np.concatenate([img, img, img], axis=2)
        img = Image.fromarray(img)
    
        
      

        if self.transform is not None:
            img = self.transform(img)
        label = self.data.iloc[index, 1:].values
        label = np.array(label, dtype=np.float32)
        label = torch.tensor(label, dtype=torch.float32)

        return img, label
# create a transform
# create a transform

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    #transforms.ToPILImage(),
   # transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    #transforms.RandomRotation(20),
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    #transforms.ToPILImage(),
   # transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((255, 255)),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# create a train and validation dataset
# Selecting the first 10 rows
data = data.iloc[:50000]

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)



train_dataset = CXR8Dataset(train_data, data_dir, train_transform)
val_dataset = CXR8Dataset(val_data, data_dir, val_transform)

# create a dataloader
if __name__ == '__main__':
    train_loader = DataLoader(
        train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=0,  # Run single-threaded to identify issues
        pin_memory=True
    )

    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)



In [19]:
train_data

,Image Index,Pleural_Thickening,Consolidation,Cardiomegaly,Nodule,Emphysema,Pneumothorax,Infiltration,Effusion,Hernia,Atelectasis,Pneumonia,Mass,Edema,Fibrosis
39087,00010238_001.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30893,00008045_005.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45278,00011606_003.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
16398,00004376_000.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13653,00003528_065.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,00002954_001.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44732,00011504_007.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38158,00010007_077.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
860,00000211_014.png,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# create an instance resnet50v2 model
import torchvision.models as models 
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.models.resnet import ResNet50_Weights
from torchvision.models.densenet import DenseNet121_Weights
model = models.densenet121(weights=DenseNet121_Weights.DEFAULT)

    #model.conv1 = nn.Conv2d(1, model.conv1.out_channels, kernel_size=model.conv1.kernel_size, stride=model.conv1.stride, padding=model.conv1.padding, bias=model.conv1.bias)
    #model.features.conv0 = nn.Conv2d(1, model.features.conv0.out_channels, kernel_size=model.features.conv0.kernel_size, stride=model.features.conv0.stride, padding=model.features.conv0.padding, bias=model.features.conv0.bias)

model.classifier = nn.Sequential(
        nn.Linear(model.classifier.in_features, 1024),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 14),
    )



#     # save the model
#    torch.save(model.state_dict(), 'model.pth')

#     # load the model
model.load_state_dict(torch.load('model.pth'))
model.to('cuda')




C:\Users\jonin\AppData\Local\Temp\ipykernel_91596\2910340252.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pth'))


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [22]:
device

device(type='cuda')

In [23]:
# evaluate the model
model.eval()
predictions = []
actuals = []
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        output = model(images)
        output = torch.sigmoid(output)
        output = output.cpu().detach().numpy()
        predictions.extend(output)
        actuals.extend(labels.cpu().detach().numpy())

predictions = np.array(predictions)
actuals = np.array(actuals)


In [24]:
predictions2 = predictions.copy()


In [25]:

predictions = predictions2

In [26]:

predictions = np.where(predictions > 0.01, 1, 0)

# calculate the metrics
accuracy = accuracy_score(actuals, predictions)
recall = recall_score(actuals, predictions, average='micro')
precision = precision_score(actuals, predictions, average='micro')
f1 = f1_score(actuals, predictions, average='micro')


In [27]:
print(classification_report(actuals, predictions, target_names=all_labels_without_no_finding, zero_division=0))


                    precision    recall  f1-score   support

Pleural_Thickening       0.03      1.00      0.06       325
     Consolidation       0.04      0.04      0.04       359
      Cardiomegaly       0.03      0.66      0.06       262
            Nodule       0.07      0.69      0.12       507
         Emphysema       0.02      0.99      0.05       210
      Pneumothorax       0.06      0.87      0.12       455
      Infiltration       0.17      0.40      0.24      1495
          Effusion       0.16      0.78      0.26      1095
            Hernia       0.00      0.93      0.01        27
       Atelectasis       0.11      0.41      0.17       941
         Pneumonia       0.01      0.86      0.03       112
              Mass       0.04      0.30      0.07       427
             Edema       0.06      0.85      0.11       189
          Fibrosis       0.02      0.99      0.04       191

         micro avg       0.05      0.59      0.09      6595
         macro avg       0.06      0.7

In [28]:
# calculate the roc auc score
from sklearn.metrics import roc_auc_score

# calculate the roc auc score for each label
roc_auc_scores = []
best_thresholds = []

for i in range(len(all_labels_without_no_finding)):
    
    best = 0
    best_threshold = 0
    for threshold in np.arange(0.001, 1, 0.001):
        predictions = predictions2.copy()
        predictions = np.where(predictions > threshold, 1, 0)
        roc_auc = roc_auc_score(actuals[:, i], predictions[:, i])
        if roc_auc > best:
            best = roc_auc
            best_threshold = threshold
    best_thresholds.append(best_threshold)
    roc_auc_scores.append([best, all_labels_without_no_finding[i]])

for i in range(len(all_labels_without_no_finding)):
    print(f'{all_labels_without_no_finding[i]}: {roc_auc_scores[i][0]}')



Pleural_Thickening: 0.6142635658914728
Consolidation: 0.5351289568489266
Cardiomegaly: 0.5647871171251679
Nodule: 0.6037761448225839
Emphysema: 0.6471642589620117
Pneumothorax: 0.650487281183981
Infiltration: 0.5625870282875114
Effusion: 0.6369639959081015
Hernia: 0.6961685439575743
Atelectasis: 0.5894908557303937
Pneumonia: 0.6243932038834952
Mass: 0.5498365695282227
Edema: 0.7988970376086878
Fibrosis: 0.6580827309464168


In [29]:
roc_auc_scores

[[0.6142635658914728, 'Pleural_Thickening'],
 [0.5351289568489266, 'Consolidation'],
 [0.5647871171251679, 'Cardiomegaly'],
 [0.6037761448225839, 'Nodule'],
 [0.6471642589620117, 'Emphysema'],
 [0.650487281183981, 'Pneumothorax'],
 [0.5625870282875114, 'Infiltration'],
 [0.6369639959081015, 'Effusion'],
 [0.6961685439575743, 'Hernia'],
 [0.5894908557303937, 'Atelectasis'],
 [0.6243932038834952, 'Pneumonia'],
 [0.5498365695282227, 'Mass'],
 [0.7988970376086878, 'Edema'],
 [0.6580827309464168, 'Fibrosis']]